In [1]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.0 MB/s eta 0:00:00


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=3372967bea8ed7a0ea7c821af2f02fc63e2416f56779ea572752d7c09b426b6c
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


## Import thư viện + data

In [3]:
import itertools
import os
import random

os.environ["DGLBACKEND"] = "pytorch"

import dgl
from dgl.data import DGLDataset
import pandas as pd
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

## Prepare Data

In [4]:
dataset_name = "link_prediction"
nodes_data_path = "/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/node.txt"
edges_data_path = "/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/egdes.txt"

In [5]:
nodes_data = pd.read_csv(nodes_data_path)
edges_data = pd.read_csv(edges_data_path)

In [6]:
# Mapping ID
id_map = {}
for i, id in enumerate(nodes_data['user'].unique()):
  id_map[id] = i
nodes_data["user"] = nodes_data["user"].apply(lambda x: id_map[x])
edges_data['Source'] = edges_data["Source"].apply(lambda x: id_map[x])
edges_data['Target'] = edges_data["Target"].apply(lambda x: id_map[x])

In [7]:
edges_data.to_csv("./edges_mapping.csv", index = False)

In [8]:
nodes_data.to_csv("./nodes_mapping.csv", index = False)

In [11]:
import os.path as osp
import random
import pandas as pd
import torch
from torch_geometric.data import InMemoryDataset, Data


class LinkPredictionDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(LinkPredictionDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ["nodes_mapping.csv", "edges_mapping.csv"]

    @property
    def processed_file_names(self):
        return ["data.pt"]

    def download(self):
        pass

    def process(self):
        nodes_data = pd.read_csv(osp.join(self.raw_dir, "nodes_mapping.csv"))
        edges_data = pd.read_csv(osp.join(self.raw_dir, "edges_mapping.csv"))
        print("Heloo")
        node_features1 = torch.from_numpy(nodes_data["degree"].to_numpy())
        print(node_features1)
        node_features2 = torch.from_numpy(nodes_data["clustering"].to_numpy())
        print(node_features2)
        node_features = torch.stack([node_features1, node_features2], dim=1)
        print(node_features)

        edges_src = torch.from_numpy(edges_data["Source"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Target"].to_numpy())

        data = Data(x=node_features, edge_index=torch.stack([edges_src, edges_dst]))

        # Create a train/test split for link prediction
        n_edges = edges_data.shape[0]
        n_train = int(n_edges * 0.8)
        train_edges = random.sample(range(n_edges), n_train)
        test_edges = list(set(range(n_edges)) - set(train_edges))
        train_mask = torch.zeros(n_edges, dtype=torch.bool)
        train_mask[train_edges] = True
        test_mask = torch.zeros(n_edges, dtype=torch.bool)
        test_mask[test_edges] = True

        # Create negative samples for the test set
        n_test_neg = len(test_edges)
        test_neg_src = []
        test_neg_dst = []
        edge_index = data.edge_index
        while len(test_neg_src) < n_test_neg:
            u, v = random.sample(range(nodes_data.shape[0]), 2)
            has_edge = ((edge_index[0] == u) & (edge_index[1] == v)).any()
            if not has_edge:
                test_neg_src.append(u)
                test_neg_dst.append(v)

        data.train_pos_edge_index = data.edge_index[:, train_mask]
        data.test_pos_edge_index = data.edge_index[:, test_mask]
        data.test_neg_edge_index = torch.stack([torch.tensor(test_neg_src), torch.tensor(test_neg_dst)])

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        torch.save(self.collate([data]), self.processed_paths[0])

if osp.exists(osp.join("./", "processed", "data.pt")):
    os.remove(osp.join("./", "processed", "data.pt"))
dataset = LinkPredictionDataset(root="./")
data = dataset[0]
dataset[0]

Processing...


Heloo
tensor([2, 4, 7,  ..., 1, 1, 1])
tensor([1.0000, 1.0000, 0.3810,  ..., 0.0000, 0.0000, 0.0000],
       dtype=torch.float64)
tensor([[2.0000, 1.0000],
        [4.0000, 1.0000],
        [7.0000, 0.3810],
        ...,
        [1.0000, 0.0000],
        [1.0000, 0.0000],
        [1.0000, 0.0000]], dtype=torch.float64)


Done!


Data(x=[58652, 2], edge_index=[2, 59289], train_pos_edge_index=[2, 47431], test_pos_edge_index=[2, 11858], test_neg_edge_index=[2, 11858])

In [12]:
data.x.shape

torch.Size([58652, 2])

## GNNs Model

In [27]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

class LinkPredictionModel(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(LinkPredictionModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = x.double()
        x = self.conv1(x, edge_index)
        # print(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        # print(x)
        return x

def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()

    # Negative sampling for link prediction
    edge_index = data.train_pos_edge_index
    neg_edge_index = negative_sampling(
        edge_index, num_nodes = data.num_nodes,
        num_neg_samples=edge_index.size(1))

    x = model(data.x, edge_index)
    pos_loss = -torch.log(torch.sigmoid((x[edge_index[0]] * x[edge_index[1]]).sum(dim=-1) + 1e-15)).mean()
    neg_loss = -torch.log(1 - torch.sigmoid((x[neg_edge_index[0]] * x[neg_edge_index[1]]).sum(dim=-1)) + 1e-15).mean()
    loss = pos_loss + neg_loss
    loss.backward()
    optimizer.step()

def test(model, data):
    model.eval()

    with torch.no_grad():
        x = model(data.x, data.edge_index)

        pos_edge_index = data.test_pos_edge_index
        neg_edge_index = data.test_neg_edge_index

        pos_score = (x[pos_edge_index[0]] * x[pos_edge_index[1]]).sum(dim=-1)
        neg_score = (x[neg_edge_index[0]] * x[neg_edge_index[1]]).sum(dim=-1)

        y_pred = torch.cat([pos_score, neg_score]).numpy()
        y_true = torch.cat([torch.ones(pos_score.size(0)), torch.zeros(neg_score.size(0))]).numpy()

        return y_pred, y_true

# Load your data object here
data = data

model = LinkPredictionModel(num_features=data.num_features,
                            hidden_channels=16,
                            num_classes=16)

model.conv1.lin.weight = torch.nn.Parameter(model.conv1.lin.weight.double())
model.conv1.bias = torch.nn.Parameter(model.conv1.bias.double())
model.conv2.lin.weight = torch.nn.Parameter(model.conv2.lin.weight.double())
model.conv2.bias = torch.nn.Parameter(model.conv2.bias.double())

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

from sklearn.metrics import roc_auc_score

for epoch in range(100):
    train(model, data, optimizer)
    y_pred, y_true = test(model, data)
    auc_score = roc_auc_score(y_true, y_pred)
    print(f'Epoch: {epoch:03d}, AUC: {auc_score:.4f}')


Epoch: 000, AUC: 0.7039
Epoch: 001, AUC: 0.7049
Epoch: 002, AUC: 0.7059
Epoch: 003, AUC: 0.7076
Epoch: 004, AUC: 0.7095
Epoch: 005, AUC: 0.7117
Epoch: 006, AUC: 0.7260
Epoch: 007, AUC: 0.8135
Epoch: 008, AUC: 0.8722
Epoch: 009, AUC: 0.8752
Epoch: 010, AUC: 0.7607
Epoch: 011, AUC: 0.7240
Epoch: 012, AUC: 0.7181
Epoch: 013, AUC: 0.7173
Epoch: 014, AUC: 0.7181
Epoch: 015, AUC: 0.7221
Epoch: 016, AUC: 0.7371
Epoch: 017, AUC: 0.7549
Epoch: 018, AUC: 0.8277
Epoch: 019, AUC: 0.8614
Epoch: 020, AUC: 0.8759
Epoch: 021, AUC: 0.8730
Epoch: 022, AUC: 0.8743
Epoch: 023, AUC: 0.8754
Epoch: 024, AUC: 0.8745
Epoch: 025, AUC: 0.8746
Epoch: 026, AUC: 0.8749
Epoch: 027, AUC: 0.8755
Epoch: 028, AUC: 0.8745
Epoch: 029, AUC: 0.8436
Epoch: 030, AUC: 0.8441
Epoch: 031, AUC: 0.8442
Epoch: 032, AUC: 0.8680
Epoch: 033, AUC: 0.8681
Epoch: 034, AUC: 0.8678
Epoch: 035, AUC: 0.8679
Epoch: 036, AUC: 0.8431
Epoch: 037, AUC: 0.8416
Epoch: 038, AUC: 0.8411
Epoch: 039, AUC: 0.8412
Epoch: 040, AUC: 0.8717
Epoch: 041, AUC:

In [14]:
model

LinkPredictionModel(
  (conv1): GCNConv(2, 16)
  (conv2): GCNConv(16, 16)
)

In [15]:
data.x

tensor([[2.0000, 1.0000],
        [4.0000, 1.0000],
        [7.0000, 0.3810],
        ...,
        [1.0000, 0.0000],
        [1.0000, 0.0000],
        [1.0000, 0.0000]], dtype=torch.float64)

In [16]:
data.edge_index

tensor([[    0,     0,     1,  ..., 42807, 42808, 42810],
        [   46, 48136, 25443,  ..., 58648, 58649, 58651]])

In [17]:
data

Data(x=[58652, 2], edge_index=[2, 59289], train_pos_edge_index=[2, 47431], test_pos_edge_index=[2, 11858], test_neg_edge_index=[2, 11858])

## Predict link

In [28]:
def predict_link(model, data, node1, node2):
    node1 = id_map[node1]
    node2 = id_map[node2]
    with torch.no_grad():
        x = model(data.x, data.edge_index)
        score = (x[node1] * x[node2]).sum()
        prob = torch.sigmoid(score)
        return prob.item()


In [29]:
model(data.x, data.edge_index)

tensor([[-0.0047,  0.4174,  0.0124,  ...,  0.0444,  0.3194,  0.8277],
        [-0.1032,  0.3061, -0.0129,  ...,  0.1203,  0.3902,  0.8118],
        [-0.1789, -0.1310, -0.0893,  ...,  0.0698, -0.0011, -0.1110],
        ...,
        [ 0.0729, -0.1352,  0.0318,  ..., -0.0457, -0.1528, -0.3325],
        [ 0.0310, -0.1718,  0.0074,  ..., -0.0328, -0.1662, -0.3818],
        [ 0.0729, -0.1352,  0.0318,  ..., -0.0457, -0.1528, -0.3325]],
       dtype=torch.float64, grad_fn=<AddBackward0>)

In [30]:
model.forward(data.x, data.edge_index).shape

torch.Size([58652, 16])

In [31]:
predict_link(model, data, 33152428, 8802740)

0.8341153605090555

In [32]:
predict_link(model, data, 112991722, 23931250)

0.5700738145936534

### Những cạnh là Fraud!

In [33]:
predict_link(model, data, 221232712, 66353306)

0.5700738145936534

In [34]:
predict_link(model, data, 235599454, 51098362)

0.5700738145936534

In [35]:
predict_link(model, data, 155985518, 133103872)

0.5700738145936534

In [36]:
predict_link(model, data, 235599454, 51098362)

0.5700738145936534